In [1]:
# Copyright 2021 Xilinx, Inc.
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Description
-----------

This demo shows how to do three things:

1. Connect to a Versal target device via ChipScope Server (cs_server) and Hardware Server (hw_server)
2. Program a Versal target device using a design PDI file
3. Examine DDR results

In [2]:
import pprint
import os
from chipscopy import get_examples_dir_or_die
from chipscopy import create_session, report_versions

# Specify locations of the running hw_server and cs_server below.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")
BOARD = os.getenv("HW_SERVER_BOARD", "vck190/production/2.0")

EXAMPLES_DIR = get_examples_dir_or_die()
PDI_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.pdi"
LTX_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.ltx"

print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PDI_FILE}")
print(f"PROBES_FILE:{LTX_FILE}")

HW_URL: TCP:xsjmdarnall40x:3121
CS_URL: xsjmdarnall40x:3042
PROGRAMMING_FILE: C:\Users\mdarnall\dev\git\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi
PROBES_FILE:C:\Users\mdarnall\dev\git\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.ltx


In [3]:
# Start of the connection
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

In [4]:
# Use the first available device and setup its debug cores
versal_device = session.devices[0]
# Program
print(f"Programming {PDI_FILE}...")
versal_device.program(PDI_FILE)

Programming C:\Users\mdarnall\dev\git\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi...


Output()

In [5]:
print(f"Discovering debug cores...")
versal_device.discover_and_setup_cores()

Discovering debug cores...


In [6]:
# print(f"Getting DDR by DDRMC Index")
# ddr = versal_device.get_ddr(0)
# props = ddr.get_property_group([])
# print(pprint.pformat(sorted(props.items()), indent=2))

ddr_list = versal_device.ddrs

# ** Getting individual DDR from the list and exercise properties **
# ddr = ddr_list.at(0)
# props = ddr.ddr_node.get_property_group([])
# print(pprint.pformat(sorted(props.items()), indent=2))

for ddr in ddr_list:
    if ddr.is_enabled:
        # Use Status string base API directly
        print(ddr.name, "is enabled.")
        print("Calibration status is: ", ddr.get_cal_status())

        # Use Property Group to get dictionary base of results
        props = ddr.ddr_node.get_property_group(["status"])
        print(pprint.pformat(props, indent=2))

        # Use get Cal Stages API directly to also get dictionary results
        props = ddr.get_cal_stages()
        print(pprint.pformat(sorted(props.items()), indent=2))

        # Use a single report command to get all latest essential
        # Status and decoded data collected as it presents
        ddr.report()
        # Specify True to argument 1, and name/path to argument 2
        # to get the report output generated and saved to a file
        ddr.report(True, "test_out.txt")
        print("Report Done.\n")
    else:
        print(ddr.name, "is NOT enabled.")

ddr_0 is enabled.
Calibration status is:  PASS
{ 'cal_error_msg': 'None',
  'cal_message': 'No errors detected during calibration.',
  'cal_status': 'PASS',
  'dqs_status': 'Running',
  'health_status': 'GOOD',
  'track_enable': 'DQS Gate Tracking, DDRMC ISR Tracking, Post-Cal Status, UB '
                  'ISR Tracking'}
[ ('cal_stage.01_F0_PHY_BISC', 'Pass'),
  ('cal_stage.02_F0_MEM_INIT', 'Pass'),
  ('cal_stage.03_F0_DQS_GATE_CAL', 'Pass'),
  ('cal_stage.04_F0_WRITE_LEVELING', 'Pass'),
  ('cal_stage.05_F0_READ_DQ_CAL', 'Pass'),
  ('cal_stage.06_F0_WRITE_DQ_DBI_CAL', 'Pass'),
  ('cal_stage.07_F0_WRITE_LATENCY_CAL', 'Pass'),
  ('cal_stage.08_F0_READ_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.09_F0_WRITE_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.10_EN_VT_TRACK', 'Pass'),
  ('cal_stage.11_READ_DQS_TRACK', 'Pass'),
  ('cal_stage.12_CAL_DONE', 'Pass')]
-------------------

 DDRMC Status 

-------------------

Calibration Status:   PASS 

Overall Health:   GOOD 

Message:   No errors d

Byte  5  Nibble 1  -   Left Margin:   138  ( 70 )  Center Point:   190  ( 96 )  Right Margin:   140  ( 71 )
Byte  6  Nibble 0  -   Left Margin:   131  ( 67 )  Center Point:   178  ( 91 )  Right Margin:   131  ( 67 )
Byte  6  Nibble 1  -   Left Margin:   129  ( 67 )  Center Point:   176  ( 91 )  Right Margin:   129  ( 67 )
Byte  7  Nibble 0  -   Left Margin:   129  ( 66 )  Center Point:   182  ( 93 )  Right Margin:   129  ( 66 )
Byte  7  Nibble 1  -   Left Margin:   129  ( 66 )  Center Point:   182  ( 93 )  Right Margin:   129  ( 66 )

 Frequency 0  - Read Margin - Complex Pattern - Falling Edge Clock in pS and (delay taps):

Byte  0  Nibble 0  -   Left Margin:   129  ( 66 )  Center Point:   166  ( 85 )  Right Margin:   129  ( 66 )
Byte  0  Nibble 1  -   Left Margin:   122  ( 64 )  Center Point:   173  ( 91 )  Right Margin:   122  ( 64 )
Byte  1  Nibble 0  -   Left Margin:   119  ( 61 )  Center Point:   170  ( 87 )  Right Margin:   121  ( 62 )
Byte  1  Nibble 1  -   Left Margin:   121  